In [28]:
"""
collect.py
"""
# coding: utf-8

from TwitterAPI import TwitterAPI
import numpy as np
import time
import sys
import csv
import pickle
import unicodecsv as csv
from pathlib import Path

consumer_key = 'VnJQm78JFzYIjcea31YGUwlET'
consumer_secret = 's51Z7hooaozCpEr2Ud26GWrYi0r54MLZy3y8LTNtq2csLrbW2l'
access_token = '791962340-uz0yFHoqS5ggX7Fgphoor9YPt5ZjXc081GQPIMMv'
access_token_secret = 'wknoEd2afXhy6ahyux4Uo2YIp3c0IViYCAtiKNr7EIVU4'

# consumer_key = 'be7UgpmTeAStsa0xvN15G9nXx'
# consumer_secret = 'XP09MsUHRR6jxKPfNiwZIkBzdUgwW9nAZJiC9YrVwpTTMCAh1o'
# access_token = '791962340-utFt0qCQlEoHrfQJikvDH18T1MM763EYXOeF1Q7v'
# access_token_secret = 'soivoSV7Ty60EIz7yx4lNN7n3pV65U9hqFg4kCZzQacDV'

# consumer_key = 'MLnZvtFFXCUmzGsguhORdKoO4'
# consumer_secret = 'oYzrEPKMJuOBLytj9wSWQLYYrqutusRTvLcjmnFgxFHTFrJykS'
# access_token = '791962340-qpHuYJl7RNxwsChrciNRwHxEGF9AHxWgSsDfwRLu'
# access_token_secret = 'HmjAUcVjwTcrscCrIZjZSArdINRgdLWVC048zgBradNoi'


def get_twitter():
    """ Construct an instance of TwitterAPI using the tokens you entered above.
    Returns:
      An instance of TwitterAPI.     
    """
    return TwitterAPI(consumer_key, consumer_secret, access_token, access_token_secret)



def get_tweets(twitter,hashtags,twts,since_id):
    
    while True:
        try:

            request = twitter.request('search/tweets',{'q':hashtags,'count':1000,'lang':'en','max_id':since_id})
            for rspittr in request:
                twts.append(rspittr)
            print (len(twts))
            return twts
        except:
            print("Unexpected error:", sys.exc_info()[0])
            break     


def get_friends(screen_name): 
    abc = sorted([id_frnds for id_frnds in twitter.request('friends/ids',{'screen_name' : screen_name, 'count' :5000})])
    return abc


def select_required(all_tweets):
    consold = []
    for one_twt in all_tweets:
        dict_twt = {}
        dict_twt['screen_name']=one_twt['user']['screen_name']
        dict_twt['userid']= one_twt['user']['id']
        dict_twt['description']=one_twt['user']['description']
        dict_twt['tweet']=one_twt['text']
        dict_twt['username']=one_twt['user']['name']
        dict_twt['since_id']=one_twt['id']
        consold.append(dict_twt)
    return consold


def write_file(tweets,filename):
    f_name=filename+'.pkl'
    print("file length after writing->",len(tweets))
    pickle.dump(tweets, open(f_name, 'wb'))

def open_file(tweets,filename):
    f_name= filename+'.pkl'
    p = Path(f_name)
    if p.is_file():
        with open(f_name, "rb") as new_fle:
            try:
                tweets = pickle.load(new_fle)
            except EOFError:
                return tweets
    return tweets    


def main():
    twitter = get_twitter()
    print('Twitter connection established')
    print('Starting the process of Collecting tweets from Twitter based on movies ')
#     hashtags=['#DubNation','#WeTheNorth','#GrindCity','#DetroitBasketball','#thunderup','#LakeShow','#DefendTheLand']

#     hashtags=['#avatar','#hacksawridge','#braveheart','#startrekbeyond','#unbroken','lalaland']

    hashtags=['#dodgers','#chicagocubs','#nationals','#brewers','#cardinals','#whitesox']
    min_id=0
    for each_tg in hashtags:
        list_of_tweets = []
        twts_file =[]
        since_id=[]
        twts_file =open_file(twts_file,each_tg)
        if twts_file:    
            for i in twts_file:
                since_id.append(i['since_id'])
            min_id=min(since_id)
            print (min_id)
        print("implies")
        print ("Length of the tweet before search ",len(twts_file))
        list_of_tweets = get_tweets(twitter,each_tg,list_of_tweets,min_id)
        list_of_tweets= select_required(list_of_tweets)
        for each_twt in list_of_tweets:
            twts_file.append(each_twt)
        write_file(twts_file,each_tg)
#     print(list_of_tweets)
    print('All Tweets saved to a file called tweets \n')
    
if __name__ == '__main__':
    main()

Twitter connection established
Starting the process of Collecting tweets from Twitter based on movies 
933051616466472960
implies
Length of the tweet before search  1286
100
file length after writing-> 1386
930476572376678400
implies
Length of the tweet before search  452
0
file length after writing-> 452
930946307409227776
implies
Length of the tweet before search  991
100
file length after writing-> 1091
933437277187002368
implies
Length of the tweet before search  194
33
file length after writing-> 227
933916524519313408
implies
Length of the tweet before search  1300
100
file length after writing-> 1400
932781873742385152
implies
Length of the tweet before search  661
100
file length after writing-> 761
All Tweets saved to a file called tweets 



In [29]:
#Cluster.py

from TwitterAPI import TwitterAPI
from pathlib import Path
import matplotlib.pyplot as plt
import pickle
import time
import sys
import copy
import networkx as nx
import math
from collections import Counter, defaultdict, deque


consumer_key = 'VnJQm78JFzYIjcea31YGUwlET'
consumer_secret = 's51Z7hooaozCpEr2Ud26GWrYi0r54MLZy3y8LTNtq2csLrbW2l'
access_token = '791962340-uz0yFHoqS5ggX7Fgphoor9YPt5ZjXc081GQPIMMv'
access_token_secret = 'wknoEd2afXhy6ahyux4Uo2YIp3c0IViYCAtiKNr7EIVU4'

# consumer_key = 'be7UgpmTeAStsa0xvN15G9nXx'
# consumer_secret = 'XP09MsUHRR6jxKPfNiwZIkBzdUgwW9nAZJiC9YrVwpTTMCAh1o'
# access_token = '791962340-utFt0qCQlEoHrfQJikvDH18T1MM763EYXOeF1Q7v'
# access_token_secret = 'soivoSV7Ty60EIz7yx4lNN7n3pV65U9hqFg4kCZzQacDV'

# consumer_key = 'MLnZvtFFXCUmzGsguhORdKoO4'
# consumer_secret = 'oYzrEPKMJuOBLytj9wSWQLYYrqutusRTvLcjmnFgxFHTFrJykS'
# access_token = '791962340-qpHuYJl7RNxwsChrciNRwHxEGF9AHxWgSsDfwRLu'
# access_token_secret = 'HmjAUcVjwTcrscCrIZjZSArdINRgdLWVC048zgBradNoi'


def get_twitter():
    """ Construct an instance of TwitterAPI using the tokens you entered above.
    Returns:
      An instance of TwitterAPI.     
    """
    return TwitterAPI(consumer_key, consumer_secret, access_token, access_token_secret)


def robust_request(twitter, screen_name, max_tries=1):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request; e.g., "friends/ids"
      params ..... A parameter dict for the request, e.g., to specify
                   parameters like screen_name or count.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """ 
    for i in range(max_tries):
        request = twitter.request('friends/ids',{'screen_name' : screen_name, 'count' :5000})
        if request.status_code == 200:
            return request
        else:
            print('Got error %s \nsleeping for 15 minutes.' % request.text)
            sys.stderr.flush()
            time.sleep(61 * 15)


def open_file(tweets,filename):
    f_name= filename+'.pkl'
    p = Path(f_name)
    if p.is_file():
        with open(f_name, "rb") as new_fle:
            try:
                tweets = pickle.load(new_fle)
            except EOFError:
                return tweets
    return tweets


def find_unique_user(tweets):
    li_usr=[]
    li_usr_id=[]
    for i in tweets:
        li_usr.append(i['screen_name'])
        li_usr_id.append(i['userid'])
    acd = list(set(li_usr))
    bcd = list(set(li_usr_id))
    return acd,bcd



def find_twitter_frnds(tweeter_names,frnds_id):
    for twtr_itr in tweeter_names[:5]:
        print (twtr_itr)
        frnds_id[twtr_itr] = sorted([id_frnds for id_frnds in robust_request(twitter,twtr_itr)])
    return frnds_id


def draw_graph(frnd):
    teams = list(frnd.keys())
    graph = nx.Graph()
    for cand in range(len(teams)):
        for each_frnd in range(cand+1,len(teams)):
            abb_tmp =list(set(frnd[teams[cand]] ) & set(frnd[teams[each_frnd]]))
            for node in abb_tmp:
                graph.add_edges_from([(teams[cand],node)]) 
                graph.add_edges_from([(teams[each_frnd],node)])
    return graph


get_ipython().magic('matplotlib inline')
def read_graph(graph,names):
    dict_lbl = {}
    pslyt = nx.spring_layout(graph)
    plt.figure(figsize=(110,110))
    nx.draw(graph, pslyt, with_labels=False,node_size =2600)
    for i_in_nde in graph.nodes():
        for each_node in names:
            if each_node == i_in_nde:
                dict_lbl[i_in_nde] = each_node
   
    nx.draw_networkx_labels(graph,pslyt,labels=dict_lbl,font_size=70,font_color='blue')
    plt.savefig("before_clusture.png")
    return graph


def girvan_newman(grph,minsize, maxsize):
    if grph.order() == 1:
        return [grph.nodes()]
    
    def edge_best(g0):
        fb = nx.edge_betweenness_centrality(g0)
        return sorted(fb.items(), key=lambda x: x[1], reverse=True)
   
    anndgrph = nx.connected_component_subgraphs(grph)
    cpnt = [each_connected for each_connected in anndgrph]
    edge_to_remove = edge_best(grph)
    citr = 0
    while len(cpnt) == 1:
        grph.remove_edge(*edge_to_remove[citr][0])
        citr = citr +1
        cpnt = [each_connected for each_connected in anndgrph]

    
    li_rslt = []
    for each_cpnt in cpnt:
        if (each_cpnt.order()>=minsize and each_cpnt.order()<=maxsize):
            li_rslt.append(each_cpnt.nodes())
        elif each_cpnt.order()>maxsize:
            li_rslt.extend(girvan_newman(each_cpnt,minsize,maxsize))
    return li_rslt



def draw_network(grph, usr, f):
    f_name="Cluster"+str(f)+".png"
    pos=nx.spring_layout(grph)    
    plt.figure(figsize=(55,45))    
    nx.draw(grph,pos,with_labels=False,node_size =3100)
    dict_labels = {}
   
    for each_nde in grph.nodes():
        for each_i in usr:
            if each_i == each_nde:
                dict_labels[each_nde] = i
    
    nx.draw_networkx_labels(grph,pos,labels=dict_labels,font_size=65,font_color='black')
    plt.savefig(f_name)



def write_file(tweets):
    f_name='Cluster.pkl'
    print("Cluster Information stored in file",f_name)
    pickle.dump(tweets, open(f_name, 'wb'))

    
def main():
    twitter = get_twitter()
    tweets=[]
    names={}   
#     hashtags=['#DubNation','#WeTheNorth','#GrindCity','#DetroitBasketball','#thunderup','#LakeShow','#DefendTheLand']
    
    
#     hashtags=['#avatar','#hacksawridge','#braveheart','#startrekbeyond','#unbroken','lalaland']
    
    hashtags=['#dodgers','#chicagocubs','#nationals','#brewers','#cardinals','#whitesox']
    count=0
    for tags in hashtags:
        tweets= open_file(tweets,tags)
        tweeter_names,tweeter_id=find_unique_user(tweets)
        count+=len(tweeter_names)
        names[tags]=tweeter_names
    teams = list(names.keys())
    for i in range(len(teams)):
        for j in range(i+1,len(teams)):
                a =list(set(names[teams[i]] ) & set(names[teams[j]]))
                print ("Mutual number of followers for the movie",teams[i],teams[j],"---->",len(a))           
 
    graph=draw_graph(names) 
    g=read_graph(graph,names)
    print('graph has %d nodes and %d edges' %(graph.order(), graph.number_of_edges()))
    result=girvan_newman(g,10, 20)
    name=1
    TEMP={}
    for i in range(1,len(result)):
        print ('cluster %d  Number of nodes/followers %d' %(i ,len(result[i])))
        draw_network(graph.subgraph(result[i]),result[i],i)
        TEMP[i]=result[i]
        TEMP['usercount']=count
    write_file(TEMP)
        
        
if __name__ == '__main__':
    main()


Mutual number of followers for the movie #dodgers #chicagocubs ----> 2
Mutual number of followers for the movie #dodgers #nationals ----> 23
Mutual number of followers for the movie #dodgers #brewers ----> 8
Mutual number of followers for the movie #dodgers #cardinals ----> 3
Mutual number of followers for the movie #dodgers #whitesox ----> 22
Mutual number of followers for the movie #chicagocubs #nationals ----> 0
Mutual number of followers for the movie #chicagocubs #brewers ----> 2
Mutual number of followers for the movie #chicagocubs #cardinals ----> 0
Mutual number of followers for the movie #chicagocubs #whitesox ----> 7
Mutual number of followers for the movie #nationals #brewers ----> 6
Mutual number of followers for the movie #nationals #cardinals ----> 1
Mutual number of followers for the movie #nationals #whitesox ----> 19
Mutual number of followers for the movie #brewers #cardinals ----> 1
Mutual number of followers for the movie #brewers #whitesox ----> 6
Mutual number of 

/Users/admin/anaconda3/lib/python3.6/site-packages/networkx/drawing/nx_pylab.py:126: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
  b = plt.ishold()
/Users/admin/anaconda3/lib/python3.6/site-packages/networkx/drawing/nx_pylab.py:138: MatplotlibDeprecationWarning: pyplot.hold is deprecated.
    Future behavior will be consistent with the long-time default:
    plot commands add elements without first clearing the
    Axes and/or Figure.
  plt.hold(b)
/Users/admin/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:917: UserWarning: axes.hold is deprecated. Please remove it from your matplotlibrc and/or style files.
  warnings.warn(self.msg_depr_set % key)
/Users/admin/anaconda3/lib/python3.6/site-packages/matplotlib/rcsetup.py:152: UserWarning: axes.hold is deprecated, will be removed in 3.0
  warnings.warn("axes.hold i

graph has 79 nodes and 159 edges
Cluster Information stored in file Cluster.pkl


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [16]:
#Classify


import pickle
from pathlib import Path
import sys
import re
import numpy as np
import time
from collections import Counter
from TwitterAPI import TwitterAPI
from collections import Counter, defaultdict, deque
import csv
from sklearn.cross_validation import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen


# In[50]:

def open_file(tweets,fname):
    fname=fname+'.pkl'
    p = Path(fname)
    if p.is_file():
        with open(fname, "rb") as file:
            try:
                tweets = pickle.load(file)
            except EOFError:
                return tweets
    return tweets


# In[51]:

def get_tweets(file,tweets_list):
    for i in file:
        t=[]
        tweets_list.append(i['tweet'])
    return tweets_list
    


# In[52]:

def process_Data(allTweet):
    data = []
    for json in allTweet:
        tweet_data = []
        tweet_data.append(json['username'])
        tweet_data.append(json['userid'])
        tweet_data.append(json['description'])
        tweet_data.append(json['tweet'])
       
        data.append(tweet_data)
    return data


# In[53]:

def write_tweets_csv(data):
    with open('twitter_data.csv', 'w',encoding='utf-8') as fp:
        a = csv.writer(fp)
        a.writerows(data)
   


# In[55]:

def affin():
    url = urlopen('http://www2.compute.dtu.dk/~faan/data/AFINN.zip')
    zipfile = ZipFile(BytesIO(url.read()))
    afinn_file = zipfile.open('AFINN/AFINN-111.txt')
    afinn = dict()
    for line in afinn_file:
        parts = line.strip().split()
        if len(parts) == 2:
            afinn[parts[0].decode("utf-8")] = int(parts[1]) 


# In[56]:

def afinn_sentiment2(terms, afinn, verbose=False):
    pos = 0
    neg = 0
    val=0
    for t in terms:
        if t in afinn:
            if verbose:
                print('\t%s=%d' % (t, afinn[t]))
            if afinn[t] > 0:
                pos += afinn[t]
                
            else:
                neg += -1 * afinn[t]
                
    val=pos+neg
    
    if val < 0 :
        return  -1
    else:
        return 1
        


# In[57]:

def create_for_manual(tweet_test,afinn):
    tweet_manual_labelling = []
    for i in tweet_test:
        i = re.sub('http\S+', 'THIS_IS_A_URL', i)
        i = re.sub('@\S+', 'THIS_IS_A_MENTION', i)
        if i.split()[0] != 'RT':
            score= afinn_sentiment2(i, afinn, verbose=False)
            
            tweet_manual_labelling.append((score,i))
    with open('tweet_manual_labelling.csv', 'w',encoding='utf-8',newline='') as fp1:
        filewriter = csv.writer(fp1)
        filewriter.writerows(tweet_manual_labelling)


# In[58]:

def write_classification(f,logistic_prediction,tweets_list):
    
    tweet_classified_labelling = []
    for i in range(len(tweets_list)):
        tweet_classified_labelling.append((logistic_prediction[i],tweets_list[i]))
   
    with open(f, 'w',encoding='utf-8',newline='') as fp1:
        filewriter = csv.writer(fp1)
        filewriter.writerows(tweet_classified_labelling)


# In[59]:

def read_tarining_data(filename):
    tweets = []
    labels1 = []
    with open(filename, 'r',encoding='utf-8') as x:
        filereader = csv.reader(x)
        for  row in filereader:
            labels1.append(row[0])
            tweets.append(row[1])
    return tweets,np.array(labels1)


# In[60]:

def tokenize(text):
    
    tokens = re.findall(r"\w+|\S", text.lower(),flags = re.L)
    tokens1 = []
    for i in tokens:
        i = re.sub('http\S+', 'THIS_IS_A_URL', i)
        i = re.sub('@\S+', 'THIS_IS_A_MENTION', i)
        x = re.findall(r"\w+|\S", i,flags = re.U)
        for j in x:
            tokens1.append(j)            
    return tokens1


# In[61]:

def do_vectorize(tokenizer_fn=tokenize, min_df=1,
                 max_df=1., binary=False, ngram_range=(1,1)):

    
    vectorizer = CountVectorizer(input = 'content', tokenizer = tokenizer_fn, min_df=min_df, 
                                     max_df=max_df, binary=True, ngram_range=ngram_range,
                                 dtype = 'int',analyzer='word',token_pattern='(?u)\b\w\w+\b',encoding='utf-8' )
    return vectorizer


# In[62]:

def get_clf():
    return LogisticRegression()


# In[63]:

def prediction(CLF,trained_CSR,trained_label,untrained_tweets_CSR):
    CLF.fit(trained_CSR,trained_label)
    predicted = CLF.predict(untrained_tweets_CSR)
    return predicted


# In[64]:

def do_cross_validation(X, y,clf, n_folds=5):
    cv = KFold(len(y), n_folds)
    accuracies = []
    for train_ind, test_ind in cv: 
        clf.fit(X[train_ind], y[train_ind])
        predictions = clf.predict(X[test_ind])
        accuracies.append(accuracy_score(y[test_ind], predictions))
    avg = np.mean(accuracies)
    return avg



# In[66]:

def main():
    hashtags=['#avatar','#hacksawridge','#braveheart','#startrekbeyond','#unbroken','lalaland']
    tweets_list=[]

    labeled_tweets,labels = read_tarining_data('tweet_manual_labelling.csv')
    clf_logistic = get_clf() 
    #get the vectorizer object
    vectorizer = do_vectorize() 
    X = vectorizer.fit_transform(tweet for tweet in labeled_tweets)
    y = np.array(labels) 
    logistic_regression_accuracy = (do_cross_validation(X, y,clf_logistic))*100
    print('Average cross validation accuracy for Logistic Regression=%.1f percentage' % (logistic_regression_accuracy))
    for tags in hashtags:
        tweets=[]
        #get tweets for each team 
        tweets= open_file(tweets,tags)
        #to store in csv process the data
        data=process_Data(tweets)
        write_tweets_csv(data)    
        #data to be tested
        tweets_list= get_tweets(tweets,tweets_list)
        #create_for_manual(tweets_list,afinn) only one -- done for you
        #Prediction for unlabelled Tweets
        test_tweet_vector = vectorizer.transform(t for t in tweets_list)
        logistic_prediction =prediction(clf_logistic,X,y,test_tweet_vector)
        fname=tags+ '.csv'
        write_classification(fname,logistic_prediction,tweets_list)

        result = dict(Counter(logistic_prediction))
   


        print ("Logistic Regression Results for team",tags)
        for i in result:
            if i == '-1':
                print ("\t Tweets aganist team\t\t\t\t",result[i])

            elif i == '0':
                print ("\t Number of advertising tweets on team \t\t%d" %result[i])

            elif i == '1':
                print ("\t Number of Tweets supporting team \t\t%d" %result[i])

        labeled_tweets,labels=read_tarining_data(fname)
        
if __name__ == '__main__':
    main()


/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


ValueError: cannot use LOCALE flag with a str pattern

In [34]:
import pickle

f = open('#brewers.pkl', 'rb')   # 'rb' for reading binary file
mydict = pickle.load(f)     
f.close()                       

print(type(mydict))

print(mydict)
# for i in mydict:
#     print(mydict)

<class 'list'>
[{'screen_name': 'cammierawr', 'userid': 818665872940302336, 'description': "If someone goes live on Twitch, I'll be sure you know about it! Don't be afraid to drop a follow, thanks! #supportingsmallstreamers #twitch #promotion", 'tweet': 'RT @Cho5en_Undead: Live now with some 7 days to die! Newbie trying to survive my first horde! #TeamSupStreamers #Brewers #Brewdaism #7dayst…', 'username': '(*≧▽≦)', 'since_id': 934131638157631488}, {'screen_name': 'X3LGaming', 'userid': 299426038, 'description': 'Mention us or use #X3LGaming for #retweets  Grow your Twitch Channel with us! https://t.co/N4MuFe7c3C For business: contact@x3lgaming.com', 'tweet': 'RT @Cho5en_Undead: Live now with some 7 days to die! Newbie trying to survive my first horde! #TeamSupStreamers #Brewers #Brewdaism #7dayst…', 'username': '#X3LCommunity [2.9K]', 'since_id': 934130801972842496}, {'screen_name': 'RSG_Retweet', 'userid': 4581898467, 'description': 'Helping Build ALL Streamer Communities! Get Retwee